In [34]:
import pandas as pd
import cvxpy

from linestar import ProjectionData

In [35]:
proj = ProjectionData()

/home/eadains/DFS/linestar.py:54: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("html.parser"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 54 of the file /home/eadains/DFS/linestar.py. To get rid of this warning, pass the additional argument 'features="html.parser"' to the BeautifulSoup constructor.

  soup = BeautifulSoup(html)


In [36]:
slate = proj.slate
positions = proj.pos_mat
teams = proj.teams
games = proj.games

In [37]:
selection = cvxpy.Variable(len(slate), boolean=True)
teams_var = cvxpy.Variable(len(teams.columns), boolean=True)
games_var = cvxpy.Variable(len(games.columns), boolean=True)

# Total salary must be less than or equal to $35,000
salary = selection @ slate["Salary"] <= 35000

# Must select players from at least 3 different teams
teams_var_con = teams_var <= selection @ teams
teams_con = cvxpy.sum(teams_var) >= 3

# Must select players from at least 2 different games
games_var_con = games_var <= selection @ games
games_con = cvxpy.sum(games_var) >= 2

# No more than 4 players, not counting the pitcher, can be selected from the same team
# First term is our selected players multiplied by a boolean array where 1's indicate non-pitcher players.
# This filters the selected players so the constraint only applies to non-pitcher players
players_teams = cvxpy.multiply(selection, (~positions["P"].astype(bool)).astype(int)) @ teams <= 4

# Must have 9 players selected
total_players = cvxpy.sum(selection) == 9

# Max and min number of players we can select for each position
# Must always have 1 pitcher, who cannot fill the UTIL position
# We can select up to 1 additional player from each other position because
# the second can fill the UTIL position
positions_max = [1, 2, 2, 2, 2, 4]
positions_min = [1, 1, 1, 1, 1, 3]
positions_max_con = selection @ positions <= positions_max
positions_min_con = selection @ positions >= positions_min

# Maxmize: Total number of fantasy points expcted for selected roster
tfp = selection @ slate["Consensus"]

In [38]:
constraints = [salary,
               teams_var_con,
               teams_con,
               games_var_con,
               games_con,
               players_teams,
               total_players,
               positions_max_con,
               positions_min_con]
problem = cvxpy.Problem(cvxpy.Maximize(tfp), constraints=constraints)
result = problem.solve(solver=cvxpy.GLPK_MI, verbose=True)

                                     CVXPY                                     
                                     v1.2.1                                    
(CVXPY) May 26 05:20:22 PM: Your problem has 226 variables, 9 constraints, and 0 parameters.
(CVXPY) May 26 05:20:22 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) May 26 05:20:22 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) May 26 05:20:22 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) May 26 05:20:22 PM: Compiling problem (target solver=GLPK_MI).
(CVXPY) May 26 05:20:22 PM: Reduction chain: FlipObjective -> Dcp2Cone -> CvxAttr2Constr -> 

In [39]:
roster = dict(zip(slate["Player"], selection.value))
roster = [name for name, select in roster.items() if select > 0]

print(f"Expected Points: {selection.value @ slate['Projection']}\n")
print("Roster:")
for x in roster:
    print(x)

Expected Points: 124.16999999999999

Roster:
Aaron Nola
Byron Buxton
Mike Trout
Juan Soto
Freddie Freeman
Javier Báez
Edwin Ríos
Justin Turner
Jonathan Schoop
